In [8]:
from ocr import OCR
from PIL import Image
import cv2
import os
import keras_ocr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob as glob
import openpyxl as op
import re

def flatten(l):
    return [item for sublist in l for item in sublist]

# Dictionary crops

In [2]:
crops={
    'densities':{},
    'faz':{}
      }
crops['densities']['table_top']={
                                'l' : 1530,
                                't' : 154,
                                'r' : 1825,
                                'b' : 540
                                }
crops['densities']['table_bot']={
                                'l' : 1540,
                                't' : 565,
                                'r' : 1820,
                                'b' : 625
                                }
crops['densities']['quality']={
                                'l' : 1116,
                                't' : 55,
                                'r' : 1200,
                                'b' : 100
                                }
crops['faz']['values']={
                            'l' : 400,
                            't' : 90,
                            'r' : 800,
                            'b' : 120
                            }


crops['faz']['quality']={
                        'l' : 1116,
                        't' : 55,
                        'r' : 1200,
                        'b' : 100
                        }


# FILES

In [3]:
#first iteration
folders_patients=glob('../data/P*')
folders_visits=glob('../data/P*/V*')
visits=sorted(
    list(
        set([folder.split("/")[3] for folder in folders_visits])
    )
)
patients=sorted(
    list(
        set([folder.split('/')[2] for folder in folders_patients]) 
    )
)

#Calm down here
# imgs={}
# for patient in patients:
#     imgs[patient]={}
#     files=glob('../data/'+patient+"/*/*")
#     files=[file for file in files if 'Zone' not in file]
#     densities=[file for file in files if ('Density' in file) ]
#     fazs=[file for file in files if ('FAZ' in file)  ]
#     imgs[patient]['densities']=densities
#     imgs[patient]['fazs']=fazs


In [4]:
#second iteration
files=glob('../data/*/*/*png')
files=[file for file in files if 'QuickVue' not in file]

imgpervisit={}
for visit in visits:
    imgpervisit[visit]=sorted([file for file in files if visit in file])



In [5]:
imgpervisit['V2']

['../data/P001/V2/P001, L eye, V2, 3.0 - Density DP.png',
 '../data/P001/V2/P001, L eye, V2, 3.0 - Density SUP.png',
 '../data/P001/V2/P001, L eye, V2, 3.0 - FAZ.png',
 '../data/P001/V2/P001, L eye, V2, 6.0 - Density DP.png',
 '../data/P001/V2/P001, L eye, V2, 6.0 - Density SUP.png',
 '../data/P001/V2/P001, L eye, V2, 6.0 - FAZ.png',
 '../data/P001/V2/P001, R eye, V2, 3.0 - Density DP.png',
 '../data/P001/V2/P001, R eye, V2, 3.0 - Density SUP.png',
 '../data/P001/V2/P001, R eye, V2, 3.0 - FAZ.png',
 '../data/P001/V2/P001, R eye, V2, 6.0 - Density DP.png',
 '../data/P001/V2/P001, R eye, V2, 6.0 - Density SUP.png',
 '../data/P001/V2/P001, R eye, V2, 6.0 - FAZ.png',
 '../data/P002/V2/P002, L eye, V2, 3.0 - Density DP.png',
 '../data/P002/V2/P002, L eye, V2, 3.0 - Density SUP.png',
 '../data/P002/V2/P002, L eye, V2, 3.0 - FAZ.png',
 '../data/P002/V2/P002, R eye, V2, 3.0 - Density DP.png',
 '../data/P002/V2/P002, R eye, V2, 3.0 - Density SUP.png',
 '../data/P002/V2/P002, R eye, V2, 3.0 - FA

In [9]:
cols=[
 'QUALITY',
 'depth_whole',
 'depth_sup hemi',
 'depth_inf hemi',
 'depth_fovea',
 'depth_parafovea',
 'depth_para_sup hemi',
 'depth_parai_inf hemi',
 'depth_para temp',
 'depth_para_sup',
 'depth_para_nasal',
 'depth_para_inf',
 'depth_perifovea',
 'depth_peri_sup hemi',
 'depth_peri_inf hemi',
 'depth_peri_tempo ',
 'depth_peri_sup',
 'depth_peri_nasal',
 'depth_peri_inf',
 'thick_whole',
 'thick_sup hemi',
 'thick_inf hemi',
 'thick_fovea',
 'thick_parafovea',
 'thick_para_sup hemi',
 'thick_parai_inf hemi',
 'thick_para temp',
 'thick_para_sup',
 'thick_para_nasal',
 'thick_para_inf',
 'thick_perifovea',
 'thick_peri_sup hemi',
 'thick_peri_inf hemi',
 'thick_peri_tempo ',
 'thick_peri_sup',
 'thick_peri_nasal',
 'thick_peri_inf',
 'grid_v_d1',
 'grid_v_d2',
 'grid_v_d3',
 'grid_v_d4',
 'grid_v_d5',
 'grid_v_d6',
 'grid_v_d7',
 'grid_v_d8',
 'grid_v_d9',
 'area (mm2)',
 'perim (mm)',
 'FD']

df=pd.DataFrame(columns=['pat','v','eye','depth']+cols) # MAKE THE COLUMNS, cols come from readin the sheet.
for file in files[3:5]:
    print(file)
    pat=file.split('/')[2]
    v=file.split('/')[3]
    eye=file.split(',')[1].strip()
    depth=file.split(',')[3].split('-')[0].strip()
    fdelete = glob('evaluate/*')
    for f in fdelete:
        os.remove(f)
    if 'Density' in file:
        im = Image.open(file)
        cropped_top= im.crop((crops['densities']['table_top']['l'],
                              crops['densities']['table_top']['t'],
                              crops['densities']['table_top']['r'],
                              crops['densities']['table_top']['b']))
        cropped_bot= im.crop((crops['densities']['table_bot']['l'],
                              crops['densities']['table_bot']['t'],
                              crops['densities']['table_bot']['r'],
                              crops['densities']['table_bot']['b']))
        cropped_qual= im.crop((crops['densities']['quality']['l'],
                               crops['densities']['quality']['t'],
                               crops['densities']['quality']['r'],
                               crops['densities']['quality']['b']))
        cropped_top.save('evaluate/cropped_top.png')
        cropped_bot.save('evaluate/cropped_bot.png')
        cropped_qual.save('evaluate/cropped_quality.png')
        ocr=OCR(image_folder="evaluate/")
        results=ocr.easyocr_model_works(visualization=False)  
        temp={   'QUALITY': np.nan,
                 'depth_whole': np.nan,
                 'depth_sup hemi': np.nan,
                 'depth_inf hemi': np.nan,
                 'depth_fovea': np.nan,
                 'depth_parafovea': np.nan,
                 'depth_para_sup hemi': np.nan,
                 'depth_parai_inf hemi': np.nan,
                 'depth_para temp': np.nan,
                 'depth_para_sup': np.nan,
                 'depth_para_nasal': np.nan,
                 'depth_para_inf': np.nan,
                 'depth_perifovea': np.nan,
                 'depth_peri_sup hemi': np.nan,
                 'depth_peri_inf hemi': np.nan,
                 'depth_peri_tempo ': np.nan,
                 'depth_peri_sup': np.nan,
                 'depth_peri_nasal': np.nan,
                 'depth_peri_inf': np.nan,
                 'thick_whole': np.nan,
                 'thick_sup hemi': np.nan,
                 'thick_inf hemi': np.nan,
                 'thick_fovea': np.nan,
                 'thick_parafovea': np.nan,
                 'thick_para_sup hemi': np.nan,
                 'thick_parai_inf hemi': np.nan,
                 'thick_para temp': np.nan,
                 'thick_para_sup': np.nan,
                 'thick_para_nasal': np.nan,
                 'thick_para_inf': np.nan,
                 'thick_perifovea': np.nan,
                 'thick_peri_sup hemi': np.nan,
                 'thick_peri_inf hemi': np.nan,
                 'thick_peri_tempo ': np.nan,
                 'thick_peri_sup': np.nan,
                 'thick_peri_nasal': np.nan,
                 'thick_peri_inf': np.nan,
                 'grid_v_d1': np.nan,
                 'grid_v_d2': np.nan,
                 'grid_v_d3': np.nan,
                 'grid_v_d4': np.nan,
                 'grid_v_d5': np.nan,
                 'grid_v_d6': np.nan,
                 'grid_v_d7': np.nan,
                 'grid_v_d8': np.nan,
                 'grid_v_d9':np.nan
                }
        #QUALITY
        qual=results[2][0][1]
        temp[list(temp.keys())[0]]=qual
        #TABLE TOP
        data_lin=np.array(results[1],dtype=object)[:,1]
        data_2d=np.split(data_lin,data_lin.shape[0]/3)
        df_top=pd.DataFrame(data_2d)
        for i,val in enumerate(df_top[0].values,start=1):
            temp[list(temp.keys())[i]]=val
        for i,val in enumerate(df_top[2].values,start=19):
            temp[list(temp.keys())[i]]=val
        #TABLE BOT
        data_lin=np.array(results[0],dtype=object)[:,1]
        data_2d=np.split(data_lin,data_lin.shape[0]/3)
        df_bot=pd.DataFrame(data_2d)
        df_bot_list=flatten(df_bot.values.tolist())
        for i,val in enumerate(df_bot_list,start=37):
            temp[list(temp.keys())[i]]=val
        temp['pat']  = pat
        temp['v']    = v
        temp['eye']  = eye
        temp['depth']= depth
        df=df.append(
            temp,
            ignore_index=True)
    
    elif 'FAZ' in file:
        im = Image.open(file)
        cropped_values= im.crop((crops['faz']['values']['l'],
                                 crops['faz']['values']['t'],
                                 crops['faz']['values']['r'],
                                 crops['faz']['values']['b']))

        cropped_qual= im.crop((crops['faz']['quality']['l'],
                               crops['faz']['quality']['t'],
                               crops['faz']['quality']['r'],
                               crops['faz']['quality']['b']))
        cropped_values.save('evaluate/cropped_values.png')
        cropped_qual.save('evaluate/cropped_quality.png')
        ocr=OCR(image_folder="evaluate/")
        results=ocr.easyocr_model_works(visualization=False)
        temp=string=' '.join([result[1] for result in results[0]]) #extracting all the ugly strings from OCR for first 3 values
        clean_3=re.findall(r"[-+]?(?:\d*\.\d+|\d+)", temp)         #regex to extract only the numbers
        last=string=' '.join([result[1] for result in results[1]]) #extract quality
        if type(last) is not list:
            last=[last]
        res_clean=clean_3+last
                
        df=df.append({
            'pat':pat,
            'v':v,
            'eye':eye,
            'depth':depth,
            'area (mm2)':res_clean[0],
            'perim (mm)':res_clean[1],
            'FD':res_clean[2],
            'QUALITY':res_clean[-1]
        },
        ignore_index=True)


../data/P003/V2/P003, R eye, V2, 6.0 - Density DP.png
IMAGES: evaluate/ cropped_bot.png
IMAGES: evaluate/ cropped_top.png
IMAGES: evaluate/ cropped_quality.png
Looking for /home/carlo/.keras-ocr/craft_mlt_25k.h5


CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Looking for /home/carlo/.keras-ocr/crnn_kurapan.h5
44.1
50.0
43.9
52.5
51.5
50.8
42.9
43.6
40.6
46.6
Whole Image
287
48.2
Superior-Hemi
290
45.1
Inferior-Hemi
284
36.1
Fovea
250
52.9
ParaFovea
329
53.7
Superior-Hemi
330
52.1
Inferior-Hemi
329
54.3
Tempo
319
5344
Superior
331
51.9
Nasal
338
52.1
Inferior
329
47.6
PeriFovea
286
49.4
Superior-Hemi
291
45.9
Inferior-Hemi
281
51.0
Tempo
271
47.7
Superior
287
49.2
Nasal
312
42.6
Inferior
274
8/10
../data/P003/V2/P003, R eye, V2, 6.0 - FAZ.png
IMAGES: evaluate/ cropped_values.png
IMAGES: evaluate/ cropped_quality.png
Looking for /home/carlo/.keras-ocr/craft_mlt_25k.h5


/tmp/ipykernel_10348/4290987626.py:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(
CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Looking for /home/carlo/.keras-ocr/crnn_kurapan.h5
FAZ (mm?): 0.254.
PERIM (mm):   1.911.
FD: 53.52
8/10


/tmp/ipykernel_10348/4290987626.py:176: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({


In [21]:
#MAYBE ITS PIVOT?
# pd.pivot_table(df,index='pat')
# df.pivot(index=pat,columns=df.columns[1:])


#kinda works but ugly 
df.iloc[0].fillna('')+df.iloc[1].fillna('')

df

pat   v    eye depth QUALITY depth_whole depth_sup hemi depth_inf hemi  \
0  P003  V2  R eye   6.0    8/10        46.6           48.2           45.1   
1  P003  V2  R eye   6.0    8/10         NaN            NaN            NaN   

  depth_fovea depth_parafovea  ... grid_v_d3 grid_v_d4 grid_v_d5 grid_v_d6  \
0        36.1            52.9  ...      43.9      52.5      51.5      50.8   
1         NaN             NaN  ...       NaN       NaN       NaN       NaN   

  grid_v_d7 grid_v_d8 grid_v_d9 area (mm2) perim (mm)     FD  
0      42.9      43.6      40.6        NaN        NaN    NaN  
1       NaN       NaN       NaN      0.254      1.911  53.52  

[2 rows x 53 columns]

pat                       P003P003
v                             V2V2
eye                     R eyeR eye
depth                       6.06.0
QUALITY                   8/108/10
depth_whole                   46.6
depth_sup hemi                48.2
depth_inf hemi                45.1
depth_fovea                   36.1
depth_parafovea               52.9
depth_para_sup hemi           53.7
depth_parai_inf hemi          52.1
depth_para temp               54.3
depth_para_sup                5344
depth_para_nasal              51.9
depth_para_inf                52.1
depth_perifovea               47.6
depth_peri_sup hemi           49.4
depth_peri_inf hemi           45.9
depth_peri_tempo              51.0
depth_peri_sup                47.7
depth_peri_nasal              49.2
depth_peri_inf                42.6
thick_whole                    287
thick_sup hemi                 290
thick_inf hemi                 284
thick_fovea                    250
thick_parafovea                329
thick_para_sup hemi 

# EXCEL

In [ ]:
wb=op.load_workbook('../data/NOPAIN TRIAL OCTA DATA EXTRACTION.xlsx')

In [ ]:
sheet_names=wb.sheetnames
ws=wb.active
ws.cell(,range="A:CS").value

In [ ]:
cols=[]
for i in range(1,98):
    temp=ws.cell(row=3,column=i).value
    if temp not in cols and type(temp) is not int:
        cols.append(temp)
    elif 'grid_v_d'+str(temp) not in cols and type(temp) is int:
        cols.append('grid_v_d'+str(temp))
cols

# Density Images

In [ ]:
# Opens a image in RGB mode
# im = Image.open(r"og_pictures/P001, L eye, V2, 6.0 - Density DP.png")
im = Image.open(r"og_pictures/P001, L eye, V2, 3.0 - Density DP.png")

# Size of the image in pixels (size of original image)
# (This is not mandatory)
width, height = im.size
# Setting the points for cropped imagee
table_top={
    'l' : 1530,
    't' : 154,
    'r' : 1825,
    'b' : 540
    }
table_bot={
    'l' : 1540,
    't' : 565,
    'r' : 1820,
    'b' : 625
    }
quality={
    'l' : 1116,
    't' : 55,
    'r' : 1200,
    'b' : 100
    }

crop_dict=table_top
cropped = im.crop((crop_dict['l'], crop_dict['t'], crop_dict['r'], crop_dict['b']))
cropped.save('evaluate/cropped.png')
ocr=OCR(image_folder="evaluate/")
results=ocr.easyocr_model_works(visualization=False)  

# display(im)
display(cropped)

# Shows the image in image viewer

In [ ]:
cropped.save('evaluate/cropped.png')

In [ ]:
ocr=OCR(image_folder="evaluate/")

if __name__ == "__main__":
#     ocr.keras_ocr_works()  #THIS WORKS TERRIBLY
    results=ocr.easyocr_model_works()  
#     ocr.pytesseract_model_works()




In [ ]:
#TOP
data_lin=np.array(results,dtype=object)[:,1]
data_2d=np.split(data_lin,data_lin.shape[0]/3)
df=pd.DataFrame(data_2d)
df

In [ ]:
#BOT
data_lin=np.array(results,dtype=object)[:,1]
data_2d=np.split(data_lin,data_lin.shape[0]/3)
df=pd.DataFrame(data_2d)
df

# FAZ

In [ ]:
# Opens a image in RGB mode
# im = Image.open(r"og_pictures/P001, L eye, V2, 6.0 - Density DP.png")
im = Image.open(r"og_pictures/P001, L eye, V2, 3.0 - FAZ.png")

# Size of the image in pixels (size of original image)
# (This is not mandatory)
width, height = im.size
# Setting the points for cropped imagee
values={
    'l' : 400,
    't' : 90,
    'r' : 800,
    'b' : 120
    }


quality={
    'l' : 1116,
    't' : 55,
    'r' : 1200,
    'b' : 100
    }

crop_dict=values
cropped = im.crop((crop_dict['l'], crop_dict['t'], crop_dict['r'], crop_dict['b']))
cropped.save('evaluate/cropped.png')
ocr=OCR(image_folder="evaluate/")
results=ocr.easyocr_model_works(visualization=False)  

display(cropped)
# display(im)

# Shows the image in image viewer

In [ ]:
results